In [1]:
import pandas as pd
import numpy as np
import re
import glob
from pathlib import Path
from datetime import datetime

## Cars24

In [2]:
a = pd.concat(pd.read_csv(f) for f in glob.glob('output/csv/20251101/cars24/raw_csv/*.csv'))
a.head(15)

,name,link_product,price,odometer,transmissions,fuel_type,car_type,update_date
0,2021 Mitsubishi Triton MY21,https://www.cars24.com.au/buy-used-mitsubishi-...,"$33,490",29k kms,Manual,Diesel,Cab Chassis,20251101
1,2021 GWM Steed MY21,https://www.cars24.com.au/buy-used-gwm-steed-c...,"$15,490",30k kms,Manual,Diesel,Cab Chassis,20251101
0,2022 Renault Arkana MY22,https://www.cars24.com.au/buy-used-renault-ark...,"$20,995",45k kms,Automatic,Petrol,Coupe,20251101
0,2020 Toyota Corolla MY20,https://www.cars24.com.au/buy-used-toyota-coro...,"$20,090",115k kms,Automatic,Petrol,Hatchback,20251101
1,2022 Hyundai i30 MY22,https://www.cars24.com.au/buy-used-hyundai-i30...,"$33,990",21k kms,Automatic,Petrol,Hatchback,20251101
2,2021 Toyota Corolla MY21,https://www.cars24.com.au/buy-used-toyota-coro...,"$27,990",29k kms,Automatic,Hybrid,Hatchback,20251101
3,2015 Volkswagen Polo MY15,https://www.cars24.com.au/buy-used-volkswagen-...,"$15,090",122k kms,Automatic,Petrol,Hatchback,20251101
4,NaN,NaN,NaN,NaN,NaN,NaN,Hatchback,20251101
5,2019 Toyota Corolla MY19,https://www.cars24.com.au/buy-used-toyota-coro...,"$22,890",64k kms,Automatic,Petrol,Hatchback,20251101
6,2023 Mazda 3 MY23,https://www.cars24.com.au/buy-used-mazda-3-car...,"$28,490",14k kms,Automatic,Petrol,Hatchback,20251101


In [3]:
a.info()
a.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 279 entries, 0 to 52
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           269 non-null    object
 1   link_product   269 non-null    object
 2   price          269 non-null    object
 3   odometer       269 non-null    object
 4   transmissions  269 non-null    object
 5   fuel_type      269 non-null    object
 6   car_type       279 non-null    object
 7   update_date    279 non-null    int64 
dtypes: int64(1), object(7)
memory usage: 19.6+ KB


,update_date
count,279.0
mean,20251101.0
std,0.0
min,20251101.0
25%,20251101.0
50%,20251101.0
75%,20251101.0
max,20251101.0


In [4]:
a = a.dropna()
a = a.drop_duplicates(subset=['link_product'])

In [5]:
""" change price to int"""
a['price'] = (a['price'].str.replace(r'[\$,]','',regex=True)).astype(int)

"""add column year made"""
a['year_made'] = a['name'].str[:4].astype(int)

""" add columm car brand"""
pattern = re.compile(r'^\d{4}\s+([a-zA-z]+)')
a['brand']=a['name'].str.extract(pattern, expand = False)

""" add column car name"""
pattern = re.compile(r'^\s*\d{4}\s+\S+\s+(.*)\s[MY]+\d{2}')
a ['name'] = a['name'].str.extract(pattern, expand = False)

""" change odometer to int"""
a['odometer'] = (a['odometer'].str.replace(r'[\s[kms]','',regex=True)).astype(int)
for idx, rows in a['odometer'].items():
    if rows > 0 and rows <= 20:
        a.loc[idx, 'odometer'] = '0 - 20,000'
    elif rows > 20 and rows <= 40:
        a.loc[idx, 'odometer'] = '20,001 - 40,000'
    elif rows > 40 and rows <= 60:
        a.loc[idx, 'odometer'] = '40,001 - 60,000'
    elif rows > 60 and rows <= 80:
        a.loc[idx, 'odometer'] = '60,001 - 80,000'
    elif rows > 80 and rows <= 100:
        a.loc[idx, 'odometer'] = '80,001 - 100,000'
    elif rows > 100 and rows <= 120:
        a.loc[idx, 'odometer'] = '100,001 - 120,000'
    elif rows > 120 and rows <= 140:
        a.loc[idx, 'odometer'] = '120,001 - 140,000'
    elif rows > 140 and rows <= 160:
        a.loc[idx, 'odometer'] = '140,001 - 160,000'
    elif rows > 160 and rows <= 180:
        a.loc[idx, 'odometer'] = '160,001 - 180,000'
    elif rows > 180 and rows <= 200:
        a.loc[idx, 'odometer'] = '180,001 - 200,000'
    else:
        a.loc[idx, 'odometer'] = '200,001+'

C:\Users\luong\AppData\Local\Temp\ipykernel_20504\4258531610.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '20,001 - 40,000' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  a.loc[idx, 'odometer'] = '20,001 - 40,000'


In [6]:
ts = datetime.now().strftime('%Y%m%d')
output = Path ('output') / 'csv' / f'{ts}' / 'cars24' / 'clean csv'
output.mkdir(parents=True, exist_ok=True)
filename= f'{ts}_cars24_clean.csv'
file_path = output / filename
a.to_csv(file_path, index=False, encoding="utf-8-sig")

## cheapcarco

In [7]:
df = pd.concat(pd.read_csv(f) for f in glob.glob('output/csv/20251101/cheapcarco/raw_csv/*.csv'))
df.head(15)

,name,link_product,price,odometer,transmissions,fuel_type,car_type,update_date
0,2010 Nissan 370Z Z34 MY10 Grey 7 Speed Sports ...,https://www.cheapcarco.com.au/used-vehicles-ro...,"$18,999","204,488 Kms",Sports Automatic,Petrol - Premium ULP,Coupe,20251101
1,2010 Nissan 370Z Z34 MY10 Grey 7 Speed Sports ...,https://www.cheapcarco.com.au/used-vehicles-ro...,"$18,999","204,488 Kms",Sports Automatic,Petrol - Premium ULP,Coupe,20251101
2,2010 Nissan 370Z Z34 MY10 Grey 7 Speed Sports ...,https://www.cheapcarco.com.au/used-vehicles-ro...,"$18,999","204,488 Kms",Sports Automatic,Petrol - Premium ULP,Coupe,20251101
3,2013 Toyota 86 ZN6 GTS White 6 Speed Manual Coupe,https://www.cheapcarco.com.au/used-vehicles-ro...,"$34,999","56,312 Kms",Manual,Petrol - Premium ULP,Coupe,20251101
4,2013 Toyota 86 ZN6 GTS White 6 Speed Manual Coupe,https://www.cheapcarco.com.au/used-vehicles-ro...,"$34,999","56,312 Kms",Manual,Petrol - Premium ULP,Coupe,20251101
5,2013 Toyota 86 ZN6 GTS White 6 Speed Manual Coupe,https://www.cheapcarco.com.au/used-vehicles-ro...,"$34,999","56,312 Kms",Manual,Petrol - Premium ULP,Coupe,20251101
6,2012 Volkswagen CC Type 3CC MY13 125TDI DSG Gr...,https://www.cheapcarco.com.au/used-vehicles-ro...,"$13,999","94,300 Kms",Sports Automatic Dual Clutch,Diesel,Coupe,20251101
7,2012 Volkswagen CC Type 3CC MY13 125TDI DSG Gr...,https://www.cheapcarco.com.au/used-vehicles-ro...,"$13,999","94,300 Kms",Sports Automatic Dual Clutch,Diesel,Coupe,20251101
8,2012 Volkswagen CC Type 3CC MY13 125TDI DSG Gr...,https://www.cheapcarco.com.au/used-vehicles-ro...,"$13,999","94,300 Kms",Sports Automatic Dual Clutch,Diesel,Coupe,20251101
9,2010 Peugeot RCZ White 6 Speed Manual Coupe,https://www.cheapcarco.com.au/used-vehicles-ro...,"$9,999","141,293 Kms",Manual,Petrol - Premium ULP,Coupe,20251101


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 684 entries, 0 to 227
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           684 non-null    object
 1   link_product   684 non-null    object
 2   price          684 non-null    object
 3   odometer       684 non-null    object
 4   transmissions  684 non-null    object
 5   fuel_type      678 non-null    object
 6   car_type       684 non-null    object
 7   update_date    684 non-null    int64 
dtypes: int64(1), object(7)
memory usage: 48.1+ KB


In [9]:
df.drop_duplicates(subset=['link_product'])
df = df.dropna()
df

,name,link_product,price,odometer,transmissions,fuel_type,car_type,update_date
0,2010 Nissan 370Z Z34 MY10 Grey 7 Speed Sports ...,https://www.cheapcarco.com.au/used-vehicles-ro...,"$18,999","204,488 Kms",Sports Automatic,Petrol - Premium ULP,Coupe,20251101
1,2010 Nissan 370Z Z34 MY10 Grey 7 Speed Sports ...,https://www.cheapcarco.com.au/used-vehicles-ro...,"$18,999","204,488 Kms",Sports Automatic,Petrol - Premium ULP,Coupe,20251101
2,2010 Nissan 370Z Z34 MY10 Grey 7 Speed Sports ...,https://www.cheapcarco.com.au/used-vehicles-ro...,"$18,999","204,488 Kms",Sports Automatic,Petrol - Premium ULP,Coupe,20251101
3,2013 Toyota 86 ZN6 GTS White 6 Speed Manual Coupe,https://www.cheapcarco.com.au/used-vehicles-ro...,"$34,999","56,312 Kms",Manual,Petrol - Premium ULP,Coupe,20251101
4,2013 Toyota 86 ZN6 GTS White 6 Speed Manual Coupe,https://www.cheapcarco.com.au/used-vehicles-ro...,"$34,999","56,312 Kms",Manual,Petrol - Premium ULP,Coupe,20251101
...,...,...,...,...,...,...,...,...
223,2015 BMW X5 F15 xDrive30d Grey 8 Speed Sports ...,https://www.cheapcarco.com.au/used-vehicles-ro...,"$19,999","142,313 Kms",Sports Automatic,Diesel,Wagon,20251101
224,2015 BMW X5 F15 xDrive30d Grey 8 Speed Sports ...,https://www.cheapcarco.com.au/used-vehicles-ro...,"$19,999","142,313 Kms",Sports Automatic,Diesel,Wagon,20251101
225,2018 MG ZS AZS1 Essence 2WD Black 6 Speed Auto...,https://www.cheapcarco.com.au/used-vehicles-ro...,"$14,999","140,001 Kms",Automatic,Petrol - Premium ULP,Wagon,20251101
226,2018 MG ZS AZS1 Essence 2WD Black 6 Speed Auto...,https://www.cheapcarco.com.au/used-vehicles-ro...,"$14,999","140,001 Kms",Automatic,Petrol - Premium ULP,Wagon,20251101


In [10]:
"""Normalize transmissions"""
df['transmissions']= df['transmissions'].str.replace('Sports Automatic Dual Clutch','Automatic')
df['transmissions']= df['transmissions'].str.replace('Sports Automatic','Automatic')
df['transmissions']= df['transmissions'].str.replace('Constant Variable','Automatic')

"""Normalize fuel_type"""
df['fuel_type']= df['fuel_type'].str.replace('Petrol - Unleaded','Petrol')
df['fuel_type']= df['fuel_type'].str.replace('Petrol - Premium ULP','Petrol')

""" change price to int"""
df['price'] = (df['price'].str.replace(r'[\$,]','',regex=True)).astype(int)

"""add column year made"""
df['year_made'] = df['name'].str[:4].astype(int)

""" add columm car brand"""
pattern = re.compile(r'^\d{4}\s+([a-zA-z]+)')
df['brand']=df['name'].str.extract(pattern, expand = False)

""" add column car name"""
pattern = re.compile(r'^\s*\d{4}\s+\S+\s+(.*)')
df ['name'] = df['name'].str.extract(pattern, expand = False)

""" change odometer to int"""
df['odometer'] = (df['odometer'].str.replace(r'[\s[Kkms,]','',regex=True)).astype(int)
for idx, rows in df['odometer'].items():
    if rows > 0 and rows <= 20:
        df.loc[idx, 'odometer'] = '0 - 20,000'
    elif rows > 20 and rows <= 40:
        df.loc[idx, 'odometer'] = '20,001 - 40,000'
    elif rows > 40 and rows <= 60:
        df.loc[idx, 'odometer'] = '40,001 - 60,000'
    elif rows > 60 and rows <= 80:
        df.loc[idx, 'odometer'] = '60,001 - 80,000'
    elif rows > 80 and rows <= 100:
        df.loc[idx, 'odometer'] = '80,001 - 100,000'
    elif rows > 100 and rows <= 120:
        df.loc[idx, 'odometer'] = '100,001 - 120,000'
    elif rows > 120 and rows <= 140:
        df.loc[idx, 'odometer'] = '120,001 - 140,000'
    elif rows > 140 and rows <= 160:
        df.loc[idx, 'odometer'] = '140,001 - 160,000'
    elif rows > 160 and rows <= 180:
        df.loc[idx, 'odometer'] = '160,001 - 180,000'
    elif rows > 180 and rows <= 200:
        df.loc[idx, 'odometer'] = '180,001 - 200,000'
    else:
        df.loc[idx, 'odometer'] = '200,001+'

ValueError: invalid literal for int() with base 10: 'P.O.A'

In [ ]:
ts = datetime.now().strftime('%Y%m%d')
output = Path ('output') / 'csv' / f'{ts}' / 'cheapcarco' / 'clean csv'
output.mkdir(parents=True, exist_ok=True)
filename= f'{ts}_cheapcarco_clean.csv'
file_path = output / filename
df.to_csv(file_path, index=False, encoding="utf-8-sig")